## NFL PLayer WAR

Create a SportsIQ Score (WAR type). This score should range from 0 - 100 scale

This will take into account a players:
- EPA / Play
- Success Rate
- CPOE (QB)
- YPRR (WR / TE)
- Pass block / run block grades (OL)
- Pressure rate (DL / EDGE)
- win rate (DL / EDGE)
- Coverage success / penalties (DB)
- Special teams snaps
- Field Goal Made / Missed (K)

In [26]:
# 1. Imports
import nflreadpy as nfl
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from typing import Dict, Any

In [27]:
# 2. Load Data

data = nfl.load_player_stats(seasons=[2025], summary_level='reg')

data = data.to_pandas()

data.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,...,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0022942,P.Rivers,Philip Rivers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2025,REG,IND,3,...,0,0,NaN,0,0,0,0,None,31.66,31.66
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,PIT,16,...,0,0,NaN,0,0,0,0,None,226.08,227.08
2,00-0023853,M.Prater,Matt Prater,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,BUF,15,...,3,0,0.938776,1,1,0,0,32,0.00,0.00
3,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,2025,REG,DEN,1,...,0,0,NaN,0,0,0,0,None,0.00,0.00
4,00-0025565,N.Folk,Nick Folk,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,NYJ,16,...,0,0,1.000000,1,1,0,0,56,0.00,0.00


In [28]:
# 2b. Load Play-by-Play Data for QB Metrics
pbp = nfl.load_pbp(seasons=[2025])

pbp = pbp.to_pandas()

# 1. Calculate Season-Wide CPOE
season_cpoe = (
    pbp
    # Filter for rows where CPOE is not Null (removes runs, penalties, etc.)
    .dropna(subset=['cpoe'])
    
    # Group by the passer
    .groupby(['passer_player_id', 'passer_player_name', 'posteam'])
    
    # Aggregate: Calculate mean CPOE and count the rows (attempts)
    .agg(
        cpoe=('cpoe', 'mean'),
        attempts=('cpoe', 'count')
    )
    .reset_index()
    
    # Rename passer_player_id to gsis_id for merging
    .rename(columns={'passer_player_id': 'gsis_id'})
)

# 2. Clean up: Filter for volume and sort
# (Optional) Keep only QBs with at least 100 passes to remove outliers
filtered_cpoe = (
    season_cpoe[season_cpoe['attempts'] >= 100]
    .sort_values(by='cpoe', ascending=False)
)

# 3. View results
print(filtered_cpoe.head())

       gsis_id passer_player_name posteam       cpoe  attempts
78  00-0039851             D.Maye      NE  10.781275       472
61  00-0037834            B.Purdy      SF   7.228501       272
45  00-0036264             J.Love      GB   5.532957       411
35  00-0034869          S.Darnold     SEA   5.238202       451
50  00-0036442           J.Burrow     CIN   4.654298       248


In [29]:
# 2c. Merge PBP metrics with player stats
# Assuming 'gsis_id' exists in data, or use 'player_id' if available
if 'gsis_id' in data.columns:
    data = data.merge(filtered_cpoe, on='gsis_id', how='left')
elif 'player_id' in data.columns:
    data = data.merge(filtered_cpoe.rename(columns={'gsis_id': 'player_id'}), on='player_id', how='left')

print(f"Merged PBP metrics. Data shape: {data.shape}")

Merged PBP metrics. Data shape: (2021, 117)


In [38]:
# 2d. Add snap counts for defensive players for the season

defensive_snaps = nfl.load_snap_counts(seasons=[2025]).to_pandas()

# 1. Calculate Season-Wide Snap Counts
defensive_snaps['player_name'] = defensive_snaps['player'].str.strip().str.lower()

season_snaps = (
    defensive_snaps
    .groupby(['player_name', 'season'], dropna=False)
    .agg(
        defensive_snaps=('defense_snaps', 'sum'),
        offensive_snaps=('offense_snaps', 'sum'),
        special_teams_snaps=('st_snaps', 'sum'),
    )
    .reset_index()
)

# 2. View results
season_snaps.head()

,player_name,season,defensive_snaps,offensive_snaps,special_teams_snaps
0,a'shawn robinson,2025,657.0,0.0,67.0
1,a.j. brown,2025,0.0,843.0,1.0
2,a.j. epenesa,2025,438.0,0.0,85.0
3,a.j. green,2025,8.0,0.0,19.0
4,aaron banks,2025,0.0,747.0,56.0


In [39]:
# 2e. Merge Snap metrics with player stats
# Assuming 'gsis_id' exists in data, or use 'player_id' if available
# 2e. Merge Snap metrics with player stats (by name + season)
data['player_name'] = data['player_name'].str.strip().str.lower()
data = data.merge(season_snaps, on=['player_name', 'season'], how='left')

print(f"Merged Snap metrics. Data shape: {data.shape}")

Merged Snap metrics. Data shape: (2021, 120)


In [40]:
data.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,...,gwfg_distance_list,fantasy_points,fantasy_points_ppr,passer_player_name,posteam,cpoe,attempts_y,defensive_snaps,offensive_snaps,special_teams_snaps
0,00-0022942,p.rivers,Philip Rivers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2025,REG,IND,3,...,None,31.66,31.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0023459,a.rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,PIT,16,...,None,226.08,227.08,A.Rodgers,PIT,0.041687,472.0,NaN,NaN,NaN
2,00-0023853,m.prater,Matt Prater,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,BUF,15,...,32,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00-0024243,m.lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,2025,REG,DEN,1,...,None,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00-0025565,n.folk,Nick Folk,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2025,REG,NYJ,16,...,56,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# 3. Define Position Scorers
# Base and specific position scorers

class PositionScorer(ABC):
    """Base class for position-specific player scoring."""
    
    def __init__(self, position: str):
        self.position = position
    
    @abstractmethod
    def compute_metrics(self, player_stats: pd.Series) -> Dict[str, float]:
        """Compute position-specific metrics. Return dict of metric_name: value."""
        pass
    
    @abstractmethod
    def get_weights(self) -> Dict[str, float]:
        """Return weights for each metric (should sum to 1.0)."""
        pass
    
    def score_player(self, player_stats: pd.Series, peer_stats: pd.DataFrame) -> float:
        """
        Score a player 0-100 relative to peer group (percentile).
        
        Args:
            player_stats: Single row of player data
            peer_stats: DataFrame of all players at this position
        
        Returns:
            Score 0-100
        """
        metrics = self.compute_metrics(player_stats)
        weights = self.get_weights()
        
        # Compute weighted score (0-1)
        weighted_score = 0.0
        for metric_name, weight in weights.items():
            if metric_name in metrics:
                # Percentile normalize within peer group for this metric
                peer_values = peer_stats.apply(
                    lambda row: self.compute_metrics(row).get(metric_name, np.nan),
                    axis=1
                )
                peer_values = peer_values.dropna()
                
                if len(peer_values) > 0:
                    percentile = (peer_values <= metrics[metric_name]).sum() / len(peer_values)
                    weighted_score += weight * percentile
        
        return max(0, min(100, weighted_score * 100))


class QBScorer(PositionScorer):
    def __init__(self):
        super().__init__("QB")
    
    def compute_metrics(self, player_stats: pd.Series) -> Dict[str, float]:
        return {
            'epa_per_play': player_stats.get('passing_epa', 0) / max(1, player_stats.get('passing_attempts', 1)),
            'cpoe': player_stats.get('cpoe', 0),  # Placeholder; actual CPOE needs more data
            'pacr': player_stats.get('pacr', 0),
        }
    
    def get_weights(self) -> Dict[str, float]:
        return {
            'epa_per_play': 0.5,
            'cpoe': 0.3,
            'pacr': 0.2,
        }


class WRScorer(PositionScorer):
    def __init__(self):
        super().__init__("WR")
    
    def compute_metrics(self, player_stats: pd.Series) -> Dict[str, float]:
        return {
            'epa_per_play': player_stats.get('receiving_epa', 0) / max(1, player_stats.get('receptions', 1)),
            'yards_per_rec': player_stats.get('receiving_yards', 0) / max(1, player_stats.get('receptions', 1)),
            'racr': player_stats.get('racr', 0),
        }
    
    def get_weights(self) -> Dict[str, float]:
        return {
            'epa_per_play': 0.5,
            'yards_per_rec': 0.2,
            'rec_first_downs': 0.3,
        }


class DLScorer(PositionScorer):
    def __init__(self):
        super().__init__("DL")
    
    def compute_metrics(self, player_stats: pd.Series) -> Dict[str, float]:
        return {
            'pressure_rate': player_stats.get('defense_tackles', 0) / max(1, player_stats.get('defense_snaps', 1)),
            'win_rate': player_stats.get('def_sacks', 0) / max(1, player_stats.get('defense_snaps', 1)),
            'tfl_per_snap': player_stats.get('def_tackles_for_loss', 0) / max(1, player_stats.get('defense_snaps', 1)),
        }
    
    def get_weights(self) -> Dict[str, float]:
        return {
            'pressure_rate': 0.4,
            'win_rate': 0.4,
            'tfl_per_snap': 0.2,
        }


class DBScorer(PositionScorer):
    def __init__(self):
        super().__init__("DB")
    
    def compute_metrics(self, player_stats: pd.Series) -> Dict[str, float]:
        return {
            'coverage_success': player_stats.get('pass_break_ups', 0) / max(1, player_stats.get('defense_snaps', 1)),
            'interception_rate': player_stats.get('interceptions', 0) / max(1, player_stats.get('defense_snaps', 1)),
            'tackle_efficiency': player_stats.get('tackles', 0) / max(1, player_stats.get('defense_snaps', 1)),
        }
    
    def get_weights(self) -> Dict[str, float]:
        return {
            'coverage_success': 0.4,
            'interception_rate': 0.3,
            'tackle_efficiency': 0.3,
        }


# Registry of position scorers
POSITION_SCORERS = {
    'QB': QBScorer(),
    'WR': WRScorer(),
    'TE': WRScorer(),  # Similar metrics to WR for now
    'DL': DLScorer(),
    'EDGE': DLScorer(),
    'LB': DBScorer(),  # Similar to DB
    'DB': DBScorer(),
    'S': DBScorer(),
    'CB': DBScorer(),
}

print(f"Loaded {len(POSITION_SCORERS)} position scorers")

Loaded 9 position scorers


In [37]:
def score_all_players(stats_df: pd.DataFrame, position_col: str = 'position') -> pd.DataFrame:
    """
    Score all players using position-specific scorers.
    
    Args:
        stats_df: DataFrame with player stats
        position_col: Name of position column
    
    Returns:
        DataFrame with 'sportsiq_score' column added
    """
    stats_df = stats_df.copy()
    stats_df['sportsiq_score'] = 0.0
    
    for position in stats_df[position_col].unique():
        if pd.isna(position):
            continue
        
        # Get scorer for this position (fallback to generic if not found)
        scorer = POSITION_SCORERS.get(str(position).upper())
        if scorer is None:
            print(f"Warning: No scorer for position {position}, skipping")
            continue
        
        # Get all players at this position for peer normalization
        position_mask = stats_df[position_col] == position
        peer_stats = stats_df[position_mask]
        
        # Score each player
        scores = []
        for idx, row in peer_stats.iterrows():
            try:
                score = scorer.score_player(row, peer_stats)
                scores.append(score)
            except Exception as e:
                print(f"Error scoring {row.get('player_name', 'unknown')}: {e}")
                scores.append(np.nan)
        
        stats_df.loc[position_mask, 'sportsiq_score'] = scores
        print(f"Scored {len(scores)} {position} players")
    
    return stats_df

# Score all players
data_scored = score_all_players(data)
data_scored[['player_name', 'position', 'sportsiq_score']].head(20)

data_scored[['player_name', 'position', 'sportsiq_score']].to_csv('nfl_player_war_2025.csv', index=False)

Scored 82 QB players
Scored 138 TE players
Scored 265 LB players
Scored 240 WR players
Scored 229 CB players
Scored 6 S players
Scored 22 DB players
Scored 5 DL players
